In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [31]:
# 不要なcolumnをdropする
# 実施検査日、study_date, accessionno, 患者ID, プリセット名称は削除
drop_list = ['実施検査日(YYYYMMDD)', 'study_date', 'ACCESSIONNO', '患者ID', 'プリセット名称']
df.drop(drop_list, axis=True, inplace=True)

In [37]:
# column名を変更する
df.rename(columns={'検査時年齢': 'age', '性別': 'gender', '身長（ｃｍ）': 'height[cm]', '体重（ｋｇ）': 'weight[kg]', '依頼科名称': 'department', '入院病棟名称': 'hospital_ward', '実施検査室名称': 'room', '撮影機種': 'modality', 
                   '部位名称': 'scan_area', '検査方法': 'scan_method'}, inplace=True)

In [47]:
# CT検査室のみ限定する
df = df[df['room'] == 'ＣＴ検査室']
df

,age,gender,height[cm],weight[kg],adult_child,department,hospital_ward,room,modality,scan_area,scan_method,scan_protocol,scan_series,kV,mA,rotation_time,CTDI,DLP
0,72,M,170.0,83.0,成人,救急科,NaN,ＣＴ検査室,Revolution,胸部〜骨盤CT,造影,5.7 P+CE Chest-Pelvis Routine,1: Plain,100.0,366.41,0.5,16.64,1352.35
1,72,M,170.0,83.0,成人,救急科,NaN,ＣＴ検査室,Revolution,胸部〜骨盤CT,造影,5.7 P+CE Chest-Pelvis Routine,2: CE,100.0,366.41,0.5,16.61,1349.55
2,85,M,171.0,58.9,成人,循環器内科,NaN,ＣＴ検査室,Revolution,胸部〜骨盤CT,単純,5.7 P+CE Chest-Pelvis Routine,1: Plain,120.0,234.59,0.5,16.66,1320.20
3,91,F,150.0,40.0,成人,脳神経外科,NaN,ＣＴ検査室,Revolution,脳CT,単純,1.7 Brain Head Routine TFI-H,1: Helical,100.0,166.63,0.8,25.56,538.61
4,91,F,150.0,40.0,成人,脳神経外科,NaN,ＣＴ検査室,Revolution,脳CT,単純,1.7 Brain Head Routine TFI-H,2: HelicalC2,100.0,166.63,0.8,25.55,538.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,28,M,187.0,78.2,成人,感染症内科,NaN,ＣＴ検査室,Revolution,頸部〜骨盤CT,造影,3.5 Neck - Pelvis Helical Routine,1: Plain,100.0,470.72,0.6,13.10,1364.91
585,28,M,187.0,78.2,成人,感染症内科,NaN,ＣＴ検査室,Revolution,頸部〜骨盤CT,造影,3.5 Neck - Pelvis Helical Routine,2: CE,100.0,470.72,0.6,13.10,1364.86
586,79,M,157.1,58.0,成人,外科,NaN,ＣＴ検査室,Revolution,胸部〜骨盤CT,Dual Energy,5.26 GSIX Chest-Pelvis CEonly,1: CEDualEnergy,140.0,400.00,0.8,15.51,1173.49
587,91,M,10.0,10.0,成人,救急科,NaN,ＣＴ検査室,Revolution,脳CT,単純,1.5 QQ Brain-Head Routine TFI-H,1: Helical,120.0,215.03,0.8,50.42,1339.98


In [301]:
def preprocess_import_data(df):
    """この関数は、読み込んだデータを整形して、列名を変更する関数です.
    
    params:
        df: 読み込んだDataFrame
        
    Return:
        df: データ整形後のDataFrame
    """
    df.drop('Unnamed: 0', axis=1, inplace=True)
    # 予測に不要な特徴量を削除する
    # 実施検査日、study_date, accessionno, 患者ID, プリセット名称は削除
    drop_list = ['実施検査日(YYYYMMDD)', 'study_date', 'ACCESSIONNO', '患者ID', 'プリセット名称', 'DLP']
    df.drop(drop_list, axis=True, inplace=True)
    
    # column名を変更する
    df.rename(columns={'検査時年齢': 'age', '性別': 'gender', '身長（ｃｍ）': 'height_cm', '体重（ｋｇ）': 'weight_kg',
                       '依頼科名称': 'department', '入院病棟名称': 'hospital_ward', '実施検査室名称': 'room', '撮影機種': 'modality', 
                       '部位名称': 'scan_area', '検査方法': 'scan_method'}, inplace=True)
    # 予測に使う装置
    df.query('modality == "Revolution"', inplace=True)
    
    # 現状ではroom, modalityは１つだけを想定しているので、dropする。
    df.drop(['room', 'modality'], axis=1, inplace=True)

    # hospital_wardのNaNは'外来'を意味する
    df.loc[df['hospital_ward'].isna(), 'hospital_ward'] = '外来'
    
    df.reset_index(inplace=True)
    df.drop('index', axis=1, inplace=True)
    

In [302]:
df = pd.concat([pd.read_excel('./scan_data/202109_all_scan_data.xlsx'), 
                pd.read_excel('./scan_data/202110_all_scan_data.xlsx'),
                pd.read_excel('./scan_data/202111_all_scan_data.xlsx'),
                pd.read_excel('./scan_data/202112_all_scan_data.xlsx')])

preprocess_import_data(df)

In [303]:
df

,age,gender,height_cm,weight_kg,adult_child,department,hospital_ward,scan_area,scan_method,scan_protocol,scan_series,kV,mA,rotation_time,CTDI
0,72,M,170.0,83.0,成人,救急科,外来,胸部〜骨盤CT,造影,5.7 P+CE Chest-Pelvis Routine,1: Plain,100.0,366.41,0.5,16.64
1,72,M,170.0,83.0,成人,救急科,外来,胸部〜骨盤CT,造影,5.7 P+CE Chest-Pelvis Routine,2: CE,100.0,366.41,0.5,16.61
2,85,M,171.0,58.9,成人,循環器内科,外来,胸部〜骨盤CT,単純,5.7 P+CE Chest-Pelvis Routine,1: Plain,120.0,234.59,0.5,16.66
3,91,F,150.0,40.0,成人,脳神経外科,外来,脳CT,単純,1.7 Brain Head Routine TFI-H,1: Helical,100.0,166.63,0.8,25.56
4,91,F,150.0,40.0,成人,脳神経外科,外来,脳CT,単純,1.7 Brain Head Routine TFI-H,2: HelicalC2,100.0,166.63,0.8,25.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2840,74,M,172.0,66.6,成人,総合診療科,５西,胸部〜骨盤CT,造影,5.7 P+CE Chest-Pelvis Routine,1: Plain,100.0,252.78,0.5,11.34
2841,74,M,172.0,66.6,成人,総合診療科,５西,胸部〜骨盤CT,造影,5.7 P+CE Chest-Pelvis Routine,2: CE,100.0,252.78,0.5,11.46
2842,67,M,167.0,64.0,成人,消化器内科,外来,胸部〜骨盤CT,造影,5.13 Aorta CTA Routine SmartPrep,1: Plain,100.0,20.03,0.6,12.48
2843,67,M,167.0,64.0,成人,消化器内科,外来,胸部〜骨盤CT,造影,5.13 Aorta CTA Routine SmartPrep,2: CTA,100.0,20.03,0.6,12.44


### まずはxgboostで試す

- 本来ならkV, mA, rotation_timeも使えないはず
- hyperparameter_tuningを組み込む
- stacking, lightgbmを試す。

In [304]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import OrdinalEncoder

In [305]:
# ラベルのエンコーディング Ordinal_encoder
oe = OrdinalEncoder()
oe.set_output(transform='pandas')
# カテゴリカラムのみ抽出して、ordinal_encoder
cat_cols = df.select_dtypes(exclude=np.number).columns.to_list()
df[cat_cols] = oe.fit_transform(df[cat_cols])

# 今回はとりあえず、kVなどの線量情報が含まれてないものは単純にdropnaしてしまう
df.dropna(inplace=True)

# データをtargetとそれ以外に分割
target = 'CTDI'
X = df.drop(target, axis=1)
y = df[target]


# train testに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [306]:
# 本来はここでhyper-parameterチューニングをする
gbr = GradientBoostingRegressor(loss='squared_error', random_state=0)
gbr.fit(X_train, y_train)

GradientBoostingRegressor(random_state=0)

In [307]:
y_pred = gbr.predict(X_test)

In [308]:
print(f'mean_absolte_error: {mean_absolute_error(y_test, y_pred)}')
print(f'mean_absolute_percentage_error: {mean_absolute_percentage_error(y_test, y_pred)}')

mean_absolte_error: 1.9964032135255116
mean_absolute_percentage_error: 0.15684738455381192


In [309]:
print(f'mean_absolte_error: {mean_absolute_error(y_test, y_pred)}')
print(f'mean_absolute_percentage_error: {mean_absolute_percentage_error(y_test, y_pred)}')

mean_absolte_error: 1.9964032135255116
mean_absolute_percentage_error: 0.15684738455381192


* とりあえず、xgboostで作成した
* 精度が良いのは、CTDI値に直結する, mA, rotation_timeなど、ほぼ答えを見ている状態だから
* 今考えているのは、kVはほぼ固定であること、rotation_timeは影響を受けないので、mAを回帰→CTDIの回帰のようにスタッキングをしたら、精度良くなるかな？

In [310]:
df_predict = pd.DataFrame({'y_test': y_test, 'y_pred': y_pred})
df_predict['diff'] = df_predict['y_test'] - df_predict['y_pred']
df_predict

,y_test,y_pred,diff
898,18.68,14.909601,3.770399
1645,27.50,29.548827,-2.048827
1918,19.13,17.742849,1.387151
104,15.44,14.265433,1.174567
2367,21.86,22.435011,-0.575011
...,...,...,...
1841,34.38,34.568495,-0.188495
538,10.22,10.398800,-0.178800
2239,11.46,12.348622,-0.888622
1888,6.60,7.381421,-0.781421
